# 03. Mock Photon Comparison

In this Notebook we compare the mock photon propagation with the existing methods like the normal flow. To do that, we generate a line detector and a couple of events that we propagate using each propagator.

In [ ]:
from ananke.configurations.presets.detector import single_line_configuration
from ananke.services.detector import DetectorBuilderService
from ananke.models.collection import Collection
from olympus.event_generation.generators import get_generator
from olympus.event_generation.photon_propagation.mock_photons import MockPhotonPropagator
from olympus.event_generation.photon_propagation.norm_flow_photons import NormalFlowPhotonPropagator
from olympus.configuration.generators import DatasetConfiguration
from olympus.configuration.photon_propagation import MockPhotonPropagatorConfiguration, NormalFlowPhotonPropagatorConfiguration

Let's define the detector next:

In [ ]:
detector_service = DetectorBuilderService()
det = detector_service.get(configuration=single_line_configuration)

In [ ]:
det.df[['module_location_x', 'module_location_y', 'module_location_z']].drop_duplicates()

Next up we generate our events:

In [ ]:
configuration = DatasetConfiguration.parse_obj({
    'generator': {
        'type': 'cascade',
        'spectrum': {
            'log_minimal_energy': 2.0,
            'log_maximal_energy': 5.5,
        }
    },
    'number_of_samples': 2
})

cascades_generator = get_generator(
    detector=det,
    configuration=configuration.generator
)

records = cascades_generator.generate_records(
    number_of_samples=configuration.number_of_samples
)

sources = cascades_generator.propagate(records)

records.df.head()

In [ ]:
sources.df.head()

Now we need our photon propagators

In [ ]:
mock_photon_propagator_configuration = MockPhotonPropagatorConfiguration(resolution=18000)

mock_photon_propagator = MockPhotonPropagator(
    detector=det,
    configuration=mock_photon_propagator_configuration
)

normal_photon_propagator_configuration = NormalFlowPhotonPropagatorConfiguration(
    shape_model_path="../../hyperion/data/photon_arrival_time_nflow_params.pickle",
    counts_model_path="../../hyperion/data/photon_arrival_time_counts_params.pickle"
)

normal_photon_propagator = NormalFlowPhotonPropagator(
    detector=det,
    configuration=normal_photon_propagator_configuration
)

Now, we want to create our collections to store things

In [ ]:

mock_collection = Collection('data/mock_photon_comparison/mock_collection.h5')
normal_collection = Collection('data/mock_photon_comparison/normal_collection.h5')

mock_collection.set_records(records)
normal_collection.set_records(records)
mock_collection.set_sources(sources)
normal_collection.set_sources(sources)

Now lets Propagate:

In [ ]:
mock_photon_propagator.propagate(mock_collection)

mock_hits = mock_collection.get_hits(records.record_ids[3])

mock_hits.df.head()

And doing the same with the normal photons.

In [ ]:
normal_photon_propagator.propagate(mock_collection)

normal_hits = normal_photon_propagator.get_hits(records.record_ids[3])

normal_hits.df.head()

And doing the same with the normal photons.

In [11]:
normal_photon_propagator.propagate(mock_collection)

normal_hits = normal_photon_propagator.get_hits(records.record_ids[3])

normal_hits.df.head()

/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)
/home/janik/projects/master/env/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


,time,string_id,module_id,pmt_id,record_id,type
0,8.689281,0,7,4,-3639979918762569235,cascade
0,61.373062,0,8,2,-3639979918762569235,cascade
0,19.784624,0,8,4,-3639979918762569235,cascade
0,15.451541,0,8,9,-3639979918762569235,cascade
0,18.989155,0,8,10,-3639979918762569235,cascade


## Comparison of the photon propagators

Now that we have all the hits we want we can compare the following cases:

1. number of hits per module
2. hit arrival times


### Number of hits per module

Let's have a look at the mock propagation:

In [12]:
aggregated_mock_hits = mock_hits.df.set_index(['string_id', 'module_id'])
aggregated_mock_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         9           211     211        211   211
          10         2738    2738       2738  2738
          11         9810    9810       9810  9810
          12         2247    2247       2247  2247
          13           66      66         66    66

In [13]:
aggregated_normal_hits = normal_hits.df.set_index(['string_id', 'module_id'])
aggregated_normal_hits.groupby(level=[0,1]).count().head()

time  pmt_id  record_id  type
string_id module_id                               
0         7             1       1          1     1
          8             7       7          7     7
          9            27      27         27    27
          10          223     223        223   223
          11         1620    1620       1620  1620

In [14]:
len(mock_hits)

15072

In [15]:
len(normal_hits)

3803